In [1]:
from pyspark.sql import SparkSession
import happybase
from pyspark import SparkContext
import pandas as pd
import json
from urllib.request import urlopen
import schedule

spark = SparkSession.builder \
    .appName("HBaseExample") \
    .getOrCreate()

sc = SparkContext.getOrCreate()
connection = happybase.Connection(host='localhost', port=9090)


In [2]:
# 指定表格名稱和列族名稱
table_name = 'Youbike'
column_family = 'info'

# 創建一個表格（如果不存在的話）
if table_name.encode() not in connection.tables():
    connection.create_table(table_name, {column_family: dict()})

In [3]:
#從台北youbike 平台抓取資料，並且寫入hbase
def fetch_data():
    spark = SparkSession.builder.getOrCreate()
  
    url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'
    jsonData = urlopen(url).read().decode('utf-8')
    rdd = spark.sparkContext.parallelize([jsonData])
    df = spark.read.json(rdd)
    
    data_dict = df.toJSON().map(lambda x: json.loads(x)).collect()
    
    connection = happybase.Connection(host='localhost', port=9090)
    table = connection.table(table_name )
    
    # Iterate through the dictionary and put the data into HBase
    for item in data_dict:
        row_key = item['sno']
        columns = {k: v for k, v in item.items() if k != 'sno'}
        # Use the row_key as the HBase row key
        row = table.row(row_key)
        for column, value in columns.items():
            # Put the data into the HBase table
            column_name = f"{column_family}:{column}"
            row[column_name] = str(value)
    
        table.put(row_key.encode('utf-8'), row)
    print("Successful!!!")


##### 每2分鐘抓取資料一次，共執行20分鐘

In [4]:
import time
# Schedule the data fetching every 5 minutes
schedule.every(2).minutes.do(fetch_data)

# Run the scheduled tasks
end_time = time.time() + 60 * 20

# 執行排程
while time.time() < end_time:
    schedule.run_pending()
    time.sleep(1)
    

Successful!!!
Successful!!!
Successful!!!
Successful!!!
Successful!!!
Successful!!!
Successful!!!
Successful!!!
Successful!!!
